In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt

# 1. Function to generate the time series dataset
def generate_time_series(N):
    np.random.seed(0)  # For reproducibility
    t = np.arange(N)
    
    # Weekly cyclicity
    weekly_cycle = 10 * np.sin(2 * np.pi * t / 7)
    
    # Exogenous variable X_t
    X_t = 5 * np.cos(2 * np.pi * t / 30) + np.random.normal(0, 1, N)
    
    # Initialize y_t
    y = np.zeros(N)
    e_t = np.random.normal(0, 0.5, N)  # Noise term
    
    # Autoregressive coefficients
    phi = [0.5, -0.3, 0.2, -0.1]
    beta = 1.5  # Coefficient for X_t
    
    # Generate y_t
    for i in range(4, N):
        y[i] = (weekly_cycle[i] +
                phi[0]*y[i-1] +
                phi[1]*y[i-2] +
                phi[2]*y[i-3] +
                phi[3]*y[i-4] +
                beta*X_t[i] +
                e_t[i])
    return pd.DataFrame({'y': y, 'X': X_t})

# Generate the dataset
N = 200  # Total number of time points
data = generate_time_series(N)

# Prepare the dataset for modeling
def prepare_dataset(data, n_lags=4, n_ahead=10):
    X, y = [], []
    for i in range(n_lags, len(data) - n_ahead):
        X.append(np.hstack([
            data['y'].values[i - n_lags:i],  # lags of y
            data['X'].values[i]              # current X_t
        ]))
        y.append(data['y'].values[i:i + n_ahead])  # next n_ahead steps
    return np.array(X), np.array(y)

X, y = prepare_dataset(data)

# Time Series Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Initialize lists to store errors
mse_native_list, mape_native_list = [], []
mse_multioutput_list, mape_multioutput_list = [], []

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # 2.a Native multioutput capability
    rf_native = RandomForestRegressor(n_estimators=100, random_state=0)
    rf_native.fit(X_train, y_train)
    y_pred_native = rf_native.predict(X_test)
    
    # 2.b Using MultiOutputRegressor
    rf_base = RandomForestRegressor(n_estimators=100, random_state=0)
    rf_multioutput = MultiOutputRegressor(rf_base)
    rf_multioutput.fit(X_train, y_train)
    y_pred_multioutput = rf_multioutput.predict(X_test)
    
    # 3. Compute MSE and MAPE
    mse_native = mean_squared_error(y_test, y_pred_native)
    mape_native = mean_absolute_percentage_error(y_test, y_pred_native)
    mse_multioutput = mean_squared_error(y_test, y_pred_multioutput)
    mape_multioutput = mean_absolute_percentage_error(y_test, y_pred_multioutput)
    
    mse_native_list.append(mse_native)
    mape_native_list.append(mape_native)
    mse_multioutput_list.append(mse_multioutput)
    mape_multioutput_list.append(mape_multioutput)

# Compute average errors
avg_mse_native = np.mean(mse_native_list)
avg_mape_native = np.mean(mape_native_list)
avg_mse_multioutput = np.mean(mse_multioutput_list)
avg_mape_multioutput = np.mean(mape_multioutput_list)

print("Native Multioutput RandomForestRegressor:")
print(f"Average MSE: {avg_mse_native:.4f}")
print(f"Average MAPE: {avg_mape_native:.4f}")

print("\nMultiOutputRegressor with RandomForestRegressor:")
print(f"Average MSE: {avg_mse_multioutput:.4f}")
print(f"Average MAPE: {avg_mape_multioutput:.4f}")


Native Multioutput RandomForestRegressor:
Average MSE: 36.4432
Average MAPE: 26.0215

MultiOutputRegressor with RandomForestRegressor:
Average MSE: 36.4997
Average MAPE: 27.5251


In [3]:
y.shape

(186, 10)